In [3]:
import cv2
import numpy as np

from twilio.rest import Client

cam = cv2.VideoCapture("video.mp4")

count1= 0
count2 = 0

detected = []

RFID = []

SPEED = []



pts = np.array([[382,389],[-50,675],[546,677],[599,399]],np.int32)
p = pts.reshape((-1,1,2))

pts2 = np.array([[680,400],[730,680],[1280,680],[930,415]])
p2 = pts2.reshape((-1,1,2))
#cv2.namedWindow("frame")
#cv2.setMouseCallback("frame",mouse_pointer)

def centers (x,y,w,h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

subtractor = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

font = cv2.FONT_HERSHEY_COMPLEX
count3 = 0

RF_IDNUMBER_FOR_LEFT = 1000

RF_IDNUMBER_FORRIGHT = 2000



text4 = "RF ID: "+str(RF_IDNUMBER_FOR_LEFT)

checking = False


while True:
    
    ret , frame = cam.read()
    grey = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(grey,(3,3),1)
    
    img_sub = subtractor.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    
    
    dilated = cv2.morphologyEx (dilat, cv2. MORPH_CLOSE , kernel)
    dilated2 = cv2.morphologyEx (dilated, cv2. MORPH_CLOSE , kernel)
    
    last = cv2.erode(dilated2,np.ones((5,5)))
    
    cv2.imshow("dilat",last)
    
    
    cv2.polylines(frame,[p],True,(170,170,0),2)
    
    cv2.polylines(frame,[p2],True,(170,170,0),2)
    
    
    #cv2.line(frame, (140, 550), (580, 550), (255,127,255), 6) 

    #cv2.line(frame,(705,600),(1200,600),(255,127,255),6)
        
        
    
    _,contours,_  = cv2.findContours(last,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    
    for (_,points) in enumerate(contours):
        (x,y,w,h) = cv2.boundingRect(points)
        
        q = (w>=80 and h>=80)
        if not q:
            continue
        
        
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
        
        cent = centers(x, y, w, h)
        
        detected.append(cent)
        cv2.circle(frame,cent,4,(0,0,255),-1)
            
        detected.append(cent)
                
        nice = x+130
        
        cv2.rectangle(frame,(x,y-45),(nice,y-2),(0,255,50),-1)
        
        cv2.rectangle(frame,(x,y-47),(nice,y),(0,0,0),2)
        
        RFID.append((x-8,y-8))
        
        speed = np.random.randint(45,65)
        
        text5 = "SPEED : "+str(speed)
        
        
        for (x,y) in detected:
            
                
                
            RFID = RFID[len(RFID)-3:len(RFID)]
            
            For_text1 = RFID[-1][0]
            For_text2 = RFID[-1][1]
            
            cv2.putText(frame,text4,(For_text1+14,For_text2),font,0.55,(255,250,255),1,cv2.LINE_AA)
            
            speed_x = RFID[-1][0]
            speed_y = RFID[-1][1]
            
            cv2.putText(frame,text5,(speed_x+14,speed_y-22),font,0.5,(255,255,255),1,cv2.LINE_AA)
            
            if (x>=135 and y<=556) and (x<=580 and y>=545):
                
                RF_IDNUMBER_FOR_LEFT = RF_IDNUMBER_FOR_LEFT+1
                text4 = "RF ID : "+str(RF_IDNUMBER_FOR_LEFT)
                
                count1 = count1+1
                
                detected.remove((x,y))
                
            if (x>=705 and y<=606) and (x<=1200 and y>=583):
                count2 = count2+1 
                detected.remove((x,y))
            
        
        
        else:
            count3 = count3+1
            
        detected = detected[len(detected)-10:len(detected)]
    
         
        
                
    text1 = "TRAFFIC IN = "+ str(count1//2)
    
    text2  = "TRAFFIC OUT = " +str(count2//2)
    
    
    
    if(count3 >=150):
        count3 = 0
        count1 = 0
        count2 = 0
        
    
    cv2.rectangle(frame,(93,73),(375,110),(0,255,25),-1)
    
    cv2.rectangle(frame,(800,73),(1110,110),(0,255,25),-1)
    
    cv2.putText(frame,text1,(100,100),font,1,(225,255,250),2,cv2.LINE_AA)
    
    cv2.putText(frame,text2,(800,100),font,1,(225,255,250),2,cv2.LINE_AA)
    
    
    cv2.rectangle(frame,(40,150),(450,210),(0,255,25),-1)
    
    acc = "NO OF CASES TODAY : " + str(25)
    
    cv2.putText(frame,acc,(45,180),font,0.8,(255,255,250),2,cv2.LINE_AA)
    
    
    
    
    file = open("text.txt",'w')
    file.write(str(abs(count1-count2)))
    
    cv2.putText(frame,"CAM : 2",(1120,50),font,0.8,(255,255,255),2,cv2.LINE_AA)
    
    cv2.putText(frame,"CALIFORNIA",(1120,80),font,0.8,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow("frame",frame)
    
    if cv2.waitKey(1)& 0xFF == ord('q'):
        break
        
        
    
cam.release()
cv2.destroyAllWindows()

account_sid = "AC0296b4e477c168d11eadebdbbec065eb"
auth_token = "5b0936e21916d36e1c9a476336d41238"
client = Client(account_sid, auth_token)

call = client.calls.create(
    to = "+917972653087",
    from_="+12052364003",
     
    url="https://handler.twilio.com/twiml/EHc3f50efd921e23297fabb8f87328efa0"
)

print(call.sid)








CA5ebbbd2252edd0cd773df8ce60d5aebc
